In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy
import numpy as np

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_uncertainty_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]


        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        uncertainties = entropy(probs_c.T)  # shape: (n_samples_in_class,)
        sorted_idx = np.argsort(uncertainties)[-n_select:]  # top-k uncertain

        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = np.concatenate(X_selected, axis=0)
    y_selected = np.concatenate(y_selected, axis=0)

    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)


_, X_1, _, y_1 = classwise_uncertainty_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.01
)


In [6]:
X_1.shape

(21485, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6 8] [     4 223292  15457]
(4, 42) (4,)
Save 3 to baseline_Def3.npy
(223292, 42) (223292,)
Save 6 to baseline_Def6.npy
(15457, 42) (15457,)
Save 8 to baseline_Def8.npy
Execution Time: 0.430120 seconds
(716259,)
[1 3 6 8] [455596 252080   7004   1579]
(455596, 42) (455596,)
Save 1 to BIM_Def1.npy
(252080, 42) (252080,)
Save 3 to BIM_Def3.npy
(7004, 42) (7004,)
Save 6 to BIM_Def6.npy
(1579, 42) (1579,)
Save 8 to BIM_Def8.npy
Execution Time: 0.952258 seconds
(716259,)
[1 3 5 6 8] [446889 254067      5  13037   2261]
(446889, 42) (446889,)
Save 1 to FGSM_Def1.npy
(254067, 42) (254067,)
Save 3 to FGSM_Def3.npy
(5, 42) (5,)
Save 5 to FGSM_Def5.npy
(13037, 42) (13037,)
Save 6 to FGSM_Def6.npy
(2261, 42) (2261,)
Save 8 to FGSM_Def8.npy
Execution Time: 0.983057 seconds
(716259,)
[1 3 6 8] [455596 252080   7004   1579]
(455596, 42) (455596,)
Save 1 to PGD_Def1.npy
(252080, 42) (252080,)
Save 3 to PGD_Def3.npy
(7004, 42) (7004,)
Save 6 to PGD_Def6.npy
(1579, 42) (1579,)
Save 8 to PG

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6 8] [     4 223690  15059]
(4, 42) (4,)
Save 3 to baseline_Def3.npy
(223690, 42) (223690,)
Save 6 to baseline_Def6.npy
(15059, 42) (15059,)
Save 8 to baseline_Def8.npy
Execution Time: 0.770437 seconds
(716259,)
[1 3 6 8] [ 52800 624042  29510   9907]
(52800, 42) (52800,)
Save 1 to BIM_Def1.npy
(624042, 42) (624042,)
Save 3 to BIM_Def3.npy
(29510, 42) (29510,)
Save 6 to BIM_Def6.npy
(9907, 42) (9907,)
Save 8 to BIM_Def8.npy
Execution Time: 3.107207 seconds
(716259,)
[1 3 6 8] [ 31073 251756 423669   9761]
(31073, 42) (31073,)
Save 1 to FGSM_Def1.npy
(251756, 42) (251756,)
Save 3 to FGSM_Def3.npy
(423669, 42) (423669,)
Save 6 to FGSM_Def6.npy
(9761, 42) (9761,)
Save 8 to FGSM_Def8.npy
Execution Time: 4.543126 seconds
(716259,)
[1 3 6 8] [ 52800 624042  29510   9907]
(52800, 42) (52800,)
Save 1 to PGD_Def1.npy
(624042, 42) (624042,)
Save 3 to PGD_Def3.npy
(29510, 42) (29510,)
Save 6 to PGD_Def6.npy
(9907, 42) (9907,)
Save 8 to PGD_Def8.npy
Execution Time: 3.994406 seconds
(7

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_ActiveLearning.csv")

In [13]:
all_attack_list

0       1   2       3    4    5       6   7       8   9    10  11  \
0   XGB1       0   0       4    0    0  223292   0   15457   0    0   0   
1   XGB1  455596   0  252080    0    0    7004   0    1579   0    0   0   
2   XGB1  446889   0  254067    0    5   13037   0    2261   0    0   0   
3   XGB1  455596   0  252080    0    0    7004   0    1579   0    0   0   
4   XGB1      88   0   22616    0  152  693238   0       2   0  163   0   
5   XGB1  239731   0  304319    3    7  164573   0    7605   0   21   0   
6   XGB1       0   0      15    0    0  669873   0   46371   0    0   0   
7   XGB1  147761   0   38077  234  168   77972   0  451847  40  160   0   
8   XGB1  459499   0  251975    0    0    4782   0       0   0    3   0   
9   XGB1  410016   0  256291    0    0   49442   0     510   0    0   0   
10   RF1       0   0       4    0    0  223690   0   15059   0    0   0   
11   RF1   52800   0  624042    0    0   29510   0    9907   0    0   0   
12   RF1   31073   0  251756    0    0  423669   0    9761   0    0   0   
13   RF1   52800   0  624042    0    0   29510   0    9907   0    0   0   
14   RF1     111   0   21987    0  137  693873   0       2   0  149   0   
15   RF1  102382   0  465542   10    0  135150   0   13175   0    0   0   
16   RF1       0   0      14    0    0  671068   0   45177   0    0   0   
17   RF1  122041   0   24377  105   10   72860   0  496848   0   18   0   
18   RF1  122096   0  572252   34    0   10648   0   11229   0    0   0   
19   RF1  184982   0  263519    0    0  257770   0    9988   0    0   0   
20   DT1       0   0       0    0    0       0   0       0   0    0   0   
21   DT1       0   0       0    0    0       0   0       0   0    0   0   
22   DT1       0   0       0    0    0       0   0       0   0    0   0   
23   DT1       0   0       0    0    0       0   0       0   0    0   0   
24   DT1       0   0       0    0    0       0   0       0   0    0   0   
25   DT1       0   0       0    0    0       0   0       0   0    0   0   
26   DT1       0   0       0    0    0       0   0       0   0    0   0   
27   DT1       0   0       0    0    0       0   0       0   0    0   0   
28   DT1       0   0       0    0    0       0   0       0   0    0   0   
29   DT1       0   0       0    0    0       0   0       0   0    0   0   

        12  
0   238753  
1   716259  
2   716259  
3   716259  
4   716259  
5   716259  
6   716259  
7   716259  
8   716259  
9   716259  
10  238753  
11  716259  
12  716259  
13  716259  
14  716259  
15  716259  
16  716259  
17  716259  
18  716259  
19  716259  
20       0  
21       0  
22       0  
23       0  
24       0  
25       0  
26       0  
27       0  
28       0  
29       0